**Objective:** 
Your task is to program an agent to find the optimal policy for navigating a labyrinth from a specified starting point to a goal point using the Value Iteration algorithm.

**Step 1: Familiarize with the Environment**
- Understand the structure of the `Labyrinth` class and how it represents the labyrinth environment, including walls, the starting point, and the goal.
- Familiarize yourself with how the `Agent` class is structured, and how it interacts with the labyrinth environment.

**Step 2: Implement Value Iteration**
- Create a function or method to implement the Value Iteration algorithm.
- You'll need to initialize a utility table with zeros and iteratively update the utilities of each state (i.e., each cell in the labyrinth) based on the Bellman equation.
- The stopping criterion for Value Iteration is when the maximum change in utility is less than a small threshold, say 0.01.
- Once the utilities have converged, use them to compute the optimal policy, which specifies the best action to take in each state.

**Step 3: Modify the Agent Class**
- Modify the `act` method of the `Agent` class to use the optimal policy derived from Value Iteration instead of taking random actions.
- Optionally, you can also modify the `update` method to incorporate any additional learning or updating you wish to implement.

**Step 4: Run the Simulation**
- Run the provided simulation loop, where the agent is placed in the labyrinth and must navigate to the goal.
- Observe how the agent's behavior changes as it learns the optimal policy.
- You might want to add some print statements or other logging to help visualize the agent's path through the labyrinth and how it improves over time.



In [1]:
import random
import time

import numpy as np


class Labyrinth:
    def __init__(self, rows, cols, walls, start, goal):
        self.grid = np.zeros((rows, cols))
        for wall in walls:
            self.grid[wall] = -1  # Assign -1 for walls
        self.start = start
        self.goal = goal
        self.current_position = start

    def reset(self):
        self.current_position = self.start
        return self.current_position

    def step(self, action):
        # Assume actions are encoded as (delta_row, delta_col)
        new_position = (
            self.current_position[0] + action[0],
            self.current_position[1] + action[1],
        )
        if self.is_valid_move(new_position):
            self.current_position = new_position
        reward = 1 if self.current_position == self.goal else 0
        return self.current_position, reward

    def is_valid_move(self, position):
        rows, cols = self.grid.shape
        # return (0 <= position[0] < rows
        #         and 0 <= position[1] < cols
        #         and self.grid[position] != -1)
        if not 0 <= position[0] < rows:
            return False
        if not 0 <= position[1] < cols:
            return False
        if self.grid[position] == -1:
            return False
        return True

    def done(self):
        return self.current_position == self.goal

    def display(self, turn, move):
        grid_copy = self.grid.copy()
        grid_copy[self.current_position] = 2
        grid_copy[self.goal] = 9
        print(f'{t=} {move=}')
        print(grid_copy)
        time.sleep(1)

In [2]:
class Agent:
    def __init__(
        self,
        environment: Labyrinth,
        *,
        gamma: float = 0.6,
    ):
        self.environment = environment
        rows, cols = labyrinth.grid.shape
        self.states = [
            (i, j) for i in range(rows) for j in range(cols)
            if labyrinth.grid[i, j] != -1
        ]
        self.values = dict.fromkeys(self.states, 0)
        self.gamma = gamma  # discount
        self.actions = [
            (0, 1),
            (0, -1),
            (1, 0),
            (-1, 0),
        ]

    def reset(self):
        # self.values = dict.fromkeys(self.states, 0)
        ...

    def act(self, state) -> tuple[int, int]:
        if random.random() < 0.2:
            return random.choice(self.actions)
        row, col = state
        best_action = None
        best_value = float('-inf')
        for action in self.actions:
            new_coord = row + action[0], col + action[1]
            if self.environment.is_valid_move(new_coord):
                value = self.values[new_coord]
                if value > best_value:
                    best_action, best_value = action, value
        if best_action is None:  # kinda unnecessary, but mypy complains
            best_action = random.choice(self.actions)
        return best_action

    def update(self, action, state, reward) -> None:
        # Labyrinth step method gives just curr state and it's reward
        # we need future states & rewards for value
        for state in self.states:
            row, col = state
            new_value = float('-inf')

            for action in self.actions:
                new_state = row + action[0], col + action[1]
                if self.environment.is_valid_move(new_state):
                    reward = 1 if new_state == self.environment.goal else 0
                    value = reward + self.gamma * self.values[new_state]
                    new_value = max(new_value, value)
                    self.values[new_state] = new_value

    # def update(self, action, state, reward) -> None:
    #     new_state = state[0] + action[0], state[1] + action[1]
    #     if self.environment.is_valid_move(new_state):
    #         self.values[state] = reward + self.gamma * self.values[new_state]


# Define labyrinth
labyrinth = Labyrinth(4, 4, {(1, 1), (2, 1), (1, 2)}, (0, 0), (3, 3))
agent = Agent(environment=labyrinth)

MAX_EPISODES = 1000
T = 100


for episode in range(MAX_EPISODES):
    state = labyrinth.reset()
    agent.reset()
    for t in range(T):
        action = agent.act(state)
        state, reward = labyrinth.step(action)
        agent.update(action, state, reward)
        labyrinth.display(t, action)
        if labyrinth.done():
            print(f'{episode=}. success')
            break

# agent.values

{(0, 0): 0.8999999999999996,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (1, 0): 0.0,
 (1, 3): 2.499999999999999,
 (2, 0): 1.4999999999999993,
 (2, 2): 2.499999999999999,
 (2, 3): 0.0,
 (3, 0): 0.0,
 (3, 1): 2.499999999999999,
 (3, 2): 0.0,
 (3, 3): 2.499999999999999}